# Predicting Optimal Challenge Usage

# Part I Modeling Expected Payoff of Successful Challenge

In [1]:
import pandas as pd

In [2]:
win_exp_table=pd.read_csv('WinExp/BigTable-Table 1.csv')

In [3]:
#function takes inning, half, runner on each base, out, run difference and outputs win prob
#call it twice to see result of challenge
def win_exp(inning,half,first,second,third,outs,run_dif):
    sit=int(str(first)+str(second)+str(third),2)+1
    sit_code=str(inning)+str(half)+str(sit)+str(outs)
    return float(win_exp_table[win_exp_table.InnBaseOut==int(sit_code)][str(run_dif)].tolist()[0].strip('%'))/100

# Part II Modeling probability of success of challenge

In [4]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
#scrape all replays from 2014 season from mlb.com
mlb14=requests.get('https://baseballsavant.mlb.com/replay?challenge_type=&year=2014&challenger=teamchallenging&team=')

In [6]:
#ditto for 15
mlb15=requests.get('https://baseballsavant.mlb.com/replay?challenge_type=&year=2015&challenger=teamchallenging&team=')

In [7]:
#ditto for 16
mlb16=requests.get('https://baseballsavant.mlb.com/replay?challenge_type=&year=2016&challenger=teamchallenging&team=')

In [8]:
#format
data=[BeautifulSoup(season.text) for season in [mlb14,mlb15,mlb16]]

/Users/moshelaster/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [9]:
#initialize lists to put data into
num=[]
date=[]
teams=[]
challenger=[]
play_type=[]
overturned=[]
pitcher=[]
batter=[]
inning=[]
description=[]

In [10]:
#iterate through data to place on lists
for season in data:
    for event in season.table.findAll('tr')[1::2]:
        num.append(event.findAll('td')[0].text)
        date.append(event.findAll('td')[1].text)
        teams.append(event.findAll('td')[2].text)
        challenger.append(event.findAll('td')[3].text)
        play_type.append(event.findAll('td')[4].text)
        overturned.append(event.findAll('td')[5].text)
        pitcher.append(event.findAll('td')[6].text)
        batter.append(event.findAll('td')[7].text)
        inning.append(event.findAll('td')[8].text)

    for event in season.table.findAll('tr')[2::2]:
        description.append(event.text)       
        
        
    

In [11]:
#create column names
titles=['num',
'date',
'teams',
'challenger',
'play_type',
'overturned',
'pitcher',
'batter',
'inning',
'description']

In [12]:
#reformat and put into df
challenges=pd.DataFrame(np.matrix([num,
date,
teams,
challenger,
play_type,
overturned,
pitcher,
batter,
inning,description]).T,columns=titles)

In [13]:
#make inning into int
challenges['inning']=pd.to_numeric(challenges['inning'])

In [14]:
#read in text from retrosheet, copied and pasted from http://retrosheet.org/Replay2016.htm and saved locally
ump_stats_head=open('mlbreplay.txt').read().split('\n')[0] #the header
ump_stats=open('mlbreplay.txt').read().split('\n')[1:] #the rest of the data

In [15]:
#wasn't formatted nicely to read in, so figured out spacing through guesswork 
#between each heading to impute the spacing in subsequent data
#inds are simply the indexes that precede a new column
inds=[10,14,18,21,45,65,87,99] 

In [16]:
#copied over header, but inserted commas bf new column
ump_stats_head_fixed=''
for num,char in enumerate(ump_stats_head):
    if num in inds:
        ump_stats_head_fixed+=','
    else:
        ump_stats_head_fixed+=char

In [17]:
#split and stripped each column title
ump_stats_head_fixed=[word.rstrip() for word in ump_stats_head_fixed.split(',')]

In [18]:
#added new column to classify if umpire challenging or manager
ump_stats_head_fixed=ump_stats_head_fixed[:4]+['Init_type']+ump_stats_head_fixed[4:]

In [19]:
#same thing as before except transformed the data into a list of lists of vals
ump_stats_fixed=[]
for line in ump_stats:
    info=''
    for num,char in enumerate(line):
        if num in inds or num==25:
            info+=','
        else:
            info+=char
    ump_stats_fixed.append(map(str.rstrip,info.split(',')))


In [20]:
#read into df
ump_table=pd.DataFrame(ump_stats_fixed,columns=ump_stats_head_fixed)

In [21]:
ump_table.head()

,Date,Tm,Opp,In,Init_type,Initiator,Umpire,Type,Ruling,Time
0,03/31/2014,ATL,MIL,6,Mgr,Fredi Gonzalez,Greg Gibson,Force play,Overturned,1:05
1,03/31/2014,WAS,NYN,10,Mgr,Matt Williams,Clint Fagan,Force play,Confirmed,1:44
2,03/31/2014,CLE,OAK,6,Ump,Mike Winters,Mike Winters,HP collision,Confirmed,1:06
3,03/31/2014,CHN,PIT,5,Mgr,Rich Renteria,Bob Davidson,Force play,Confirmed,1:39
4,03/31/2014,PIT,CHN,10,Mgr,Clint Hurdle,Bob Davidson,Tag play,Overturned,1:46


In [22]:
#stripped out whitespace
for col in ump_table:
    try:
        ump_table[col].apply(lambda x: x.rstrip(),axis=1)
    except:
        pass

In [23]:
#created dictionary to map full name of team to abbreviation, to merge together both datasets
team_abbrev={
    'Giants':'SFN',
    'Angels':'ANA',
    'Astros':'HOU',
    'Athletics':'OAK',
    'Blue Jays':'TOR',
    'Braves':'ATL',
    'Brewers':'MIL',
    'Cardinals':'SLN',
    'Cubs':'CHN',
    'D-backs':'ARI',
    'Dodgers':'LAN',
    'Indians':'CLE',
    'Mariners':'SEA',
    'Marlins':'MIA',
    'Mets':'NYN',
    'Nationals':'WAS',
    'Orioles':'BAL',
    'Padres':'SDN',
    'Phillies':'PHI',
    'Pirates':'PIT',
    'Rangers':'TEX',
    'Rays':'TBA',
    'Red Sox':'BOS',
    'Reds':'CIN',
    'Rockies':'COL',
    'Royals':'KCA',
    'Tigers':'DET',
    'Twins':'MIN',
    'White Sox':'CHA',
    'Yankees':'NYA',
    'Umpire':'UMP',
    'NL':'NL'
    
}

In [24]:
#reverse the dicitonary to enable mapping back and forth
inv_teams = {team_abbr: team for team, team_abbr in team_abbrev.items()}

In [25]:
#make the changes to abbrev.
challenges.challenger=challenges.challenger.apply(lambda x: team_abbrev[x])

In [26]:
ump_table.head()

,Date,Tm,Opp,In,Init_type,Initiator,Umpire,Type,Ruling,Time
0,03/31/2014,ATL,MIL,6,Mgr,Fredi Gonzalez,Greg Gibson,Force play,Overturned,1:05
1,03/31/2014,WAS,NYN,10,Mgr,Matt Williams,Clint Fagan,Force play,Confirmed,1:44
2,03/31/2014,CLE,OAK,6,Ump,Mike Winters,Mike Winters,HP collision,Confirmed,1:06
3,03/31/2014,CHN,PIT,5,Mgr,Rich Renteria,Bob Davidson,Force play,Confirmed,1:39
4,03/31/2014,PIT,CHN,10,Mgr,Clint Hurdle,Bob Davidson,Tag play,Overturned,1:46


In [27]:
#create new column in umpire dataset to impute who challenged call
ump_table['challenger2']=np.where(ump_table['Init_type']=='Ump','UMP',ump_table['Tm'])

In [28]:
#map the play category from one dataset to another
play_dict={
    'catch or drop':'Catch/no catch',
 'fair or foul in outfield':'Fair/foul (outfield)',
 'fan interference':'Fan interference',
 'force play':'Force play',
 'grounds rule':'Grounds rule',
 'hit by pitch':'Hit by pitch',
 'home run':'Home run',
 'home-plate collision':'HP collision',
 'other':'Other',
 'passing runners':'Passing runners',
 'play at 1st':'Force play',
 'record keeping':'Rcd keeping',
 'rules check':'Rules check',
 'slide interference':'Slide rule',
 'stadium boundary call':'Boundary call',
 'tag play':'Tag play',
 'tag-up play':'Tag-up',
 'timing play':'Timing play',
 'touching a base':'Touching a base',
 'trap play':'Trap play (outfield)'
}

In [29]:
challenges.play_type=challenges.play_type.apply(lambda x: play_dict[x])

In [30]:
#change innings to num
ump_table.In=pd.to_numeric(ump_table.In)


In [31]:
#change datae to datetime
challenges.date=pd.to_datetime(challenges.date)

In [32]:
ump_table.Date=pd.to_datetime(ump_table.Date)

In [33]:
#eliminates duplicate rows, which were found in mlb.com dataset
challenges=challenges.drop_duplicates(['date','description'])

In [34]:
#modify mlb.com df to drop out the more recent challenges that are not yet recorded by retrosheet
short_challenges=challenges[challenges.date<=sorted(ump_table.Date)[-1]]

In [35]:
#trying this combination
inner_merge=ump_table.merge(short_challenges,how='inner', left_on=['challenger2','In','Date'], right_on=['challenger','inning','date']).sort_values('Date')

In [36]:
#make into day of week variable (sabr says important)
inner_merge['day']=inner_merge.date.dt.dayofweek

In [37]:
#standardize the play call labels
inner_merge.Ruling=inner_merge.Ruling.apply(lambda x: x.rstrip())

In [38]:
#get rid of retrosheet's 'Rcd keeping' box and default to mlb's ruling (assuming 0=confirmed)
inner_merge['ruling_res']=np.where(inner_merge.Ruling=='Rcd keeping', inner_merge.overturned,inner_merge.Ruling)

In [39]:
inner_merge = inner_merge[inner_merge.challenger2 != 'NL']

In [40]:
inner_merge.ruling_res=inner_merge.ruling_res.apply(lambda x: 'Confirmed' if x==0 else 'Overturned' if x==1 else x)

In [41]:
#inputs a feature in the inner_merge dataframe and calculates the 3 part breakdown of accuracy given the feature, 
#returns a dict with the unique features as keys 
def accuracy(feature):
    accuracy_dict={}
    unique_vars=feature.unique()
    for case in unique_vars:
        total=len(inner_merge[feature==case])
        overturned=len(inner_merge[(feature==case)&(inner_merge.ruling_res=='Overturned')])
        stands=len(inner_merge[(feature==case)&(inner_merge.ruling_res=='Stands')])
        confirmed=len(inner_merge[(feature==case)&(inner_merge.ruling_res=='Confirmed')])
        accuracy_dict[case]=[float(overturned)/total,float(stands)/total,float(confirmed)/total,total]
        
    return accuracy_dict


In [42]:
#create dictionaries that break down the distribution of overturned/stands/confirmed for each unique subcategory
challenger_dict=accuracy(inner_merge.challenger2)
umpire_dict=accuracy(inner_merge.Umpire)
play_dict=accuracy(inner_merge.play_type)
day_dict=accuracy(inner_merge.day)
inning_dict=accuracy(inner_merge.inning)

In [43]:
#create new vals for accuracy given each situation
inner_merge['team_overturned']=inner_merge.challenger2.apply(lambda x: challenger_dict[x][0])
inner_merge['team_stands']=inner_merge.challenger2.apply(lambda x: challenger_dict[x][1])
inner_merge['team_confirmed']=inner_merge.challenger2.apply(lambda x: challenger_dict[x][2])
inner_merge['ump_ov']=inner_merge.Umpire.apply(lambda x: umpire_dict[x][0])
inner_merge['ump_st']=inner_merge.Umpire.apply(lambda x: umpire_dict[x][1])
inner_merge['ump_con']=inner_merge.Umpire.apply(lambda x: umpire_dict[x][2])
inner_merge['type_ov']=inner_merge.play_type.apply(lambda x: play_dict[x][0])
inner_merge['type_st']=inner_merge.play_type.apply(lambda x: play_dict[x][1])
inner_merge['type_con']=inner_merge.play_type.apply(lambda x: play_dict[x][2])
inner_merge['day_ov']=inner_merge.day.apply(lambda x: day_dict[x][0])
inner_merge['day_st']=inner_merge.day.apply(lambda x: day_dict[x][1])
inner_merge['day_con']=inner_merge.day.apply(lambda x: day_dict[x][2])
inner_merge['inn_ov']=inner_merge.inning.apply(lambda x: inning_dict[x][0])
inner_merge['inn_st']=inner_merge.inning.apply(lambda x: inning_dict[x][1])
inner_merge['inn_con']=inner_merge.inning.apply(lambda x: inning_dict[x][2])

In [44]:
inner_merge['labels']=inner_merge.overturned.apply(lambda x: 1 if x=='Yes' else 0)

In [45]:
inner_merge.columns

Index([u'Date', u'Tm', u'Opp', u'In', u'Init_type', u'Initiator', u'Umpire',
       u'Type', u'Ruling', u'Time', u'challenger2', u'num', u'date', u'teams',
       u'challenger', u'play_type', u'overturned', u'pitcher', u'batter',
       u'inning', u'description', u'day', u'ruling_res', u'team_overturned',
       u'team_stands', u'team_confirmed', u'ump_ov', u'ump_st', u'ump_con',
       u'type_ov', u'type_st', u'type_con', u'day_ov', u'day_st', u'day_con',
       u'inn_ov', u'inn_st', u'inn_con', u'labels'],
      dtype='object')

In [46]:
#used for modeling on team challenging
X=inner_merge.iloc[:,-16:-1]

In [47]:
#used when exclude team history
X2=inner_merge.iloc[:,-13:-1]

In [48]:
y=inner_merge['labels']

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,log_loss
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [50]:
log_reg=LogisticRegression()

In [51]:
cross_val_score(log_reg,X2,y,scoring='log_loss').mean()

-0.62070331555903013

In [52]:
log_reg.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [53]:
log_reg2=LogisticRegression()

In [54]:
log_reg2.fit(X2,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
cross_val_score(log_reg,X,y,scoring='log_loss').mean()

-0.61496746799969715

In [56]:
log_reg.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [57]:
from  sklearn.svm import SVC

In [58]:
from sklearn.grid_search import GridSearchCV

In [59]:
svm=SVC(probability=True)

In [60]:
param_grid = [{'kernel': ['rbf'], 'gamma': [1,1e-1,1e-2,1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},{'kernel': ['linear'], 'C': [1, 10, 100, 1000]},{'kernel':['poly'],'degree':[1,2,3]}]

In [61]:
grid = GridSearchCV(svm, param_grid, scoring='log_loss')
grid.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}, {'kernel': ['poly'], 'degree': [1, 2, 3]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [62]:
print grid.best_score_
print grid.best_params_

-0.612830675541
{'kernel': 'linear', 'C': 1000}


In [63]:
grid = GridSearchCV(svm, param_grid, scoring='log_loss')
grid.fit(X2, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}, {'kernel': ['poly'], 'degree': [1, 2, 3]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='log_loss', verbose=0)

In [64]:
print grid.best_score_
print grid.best_params_

-0.620813670977
{'kernel': 'rbf', 'C': 1000, 'gamma': 0.001}


In [65]:
svm=SVC(kernel='linear',C=1000,probability=True)

In [66]:
cross_val_score(svm,X,y,scoring='log_loss').mean()

-0.61323543585579199

In [67]:
svm.fit(X,y)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [68]:
from sklearn.linear_model import LogisticRegressionCV

In [69]:
logregcv=LogisticRegressionCV()

In [70]:
cross_val_score(logregcv,X,y,scoring='log_loss').mean()

-0.61596941619059864

In [71]:
cross_val_score(logregcv,X2,y,scoring='log_loss').mean()

-0.62203500049317484

In [72]:
cross_val_score(logregcv,X2,y).mean()

0.64646851905310232

In [73]:
logregcv2=LogisticRegressionCV()

In [74]:
logregcv2.fit(X2,y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [75]:
logregcv.fit(X,y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

Now, the model will output expected probability. Now we have to convert that value taking into consideration that we are modeling on all "challengeable" plays. To model the total number of reasonably "challengeable" plays find the max for any team. To model the total "successes" find the max of successes for any team. (Maybe come back to it, but find the umpire challenges also).

In [76]:
team_games={'ANA': 3,
 'ARI': 0,
 'ATL': 0,
 'BAL': 7,
 'BOS': 0,
 'CHA': 0,
 'CHN': 9,
 'CIN': 0,
 'CLE': 0,
 'COL': 0,
 'DET': 3,
 'HOU': 6,
 'KCA': 31,
 'LAN': 9,
 'MIA': 0,
 'MIL': 0,
 'MIN': 0,
 'NYA': 1,
 'NYN': 14,
 'OAK': 1,
 'PHI': 0,
 'PIT': 2,
 'SDN': 0,
 'SEA': 0,
 'SFN': 16,
 'SLN': 13,
 'TBA': 0,
 'TEX': 5,
 'TOR': 11,
 'UMP': 0,
 'WAS': 4}

In [77]:
#calculating total umpire games
162*2*30/2+27*3*30/2+25*30/2+67

6517

In [78]:
#calculate games played, 162 for 2 + 106 from 2016 (through 7/31) plus playoffs, ump=6515 from previous summing
inner_merge['games_played']=inner_merge.challenger2.apply(lambda x: 162*2+106+team_games[x] if x!='UMP' else 6517)

In [79]:
#total challenges for a team
inner_merge['tot_challenges']=inner_merge.challenger2.apply(lambda x: len(inner_merge[inner_merge.challenger2==x]))

In [80]:
#divided by total games played
inner_merge['challenge_p_g']=inner_merge.tot_challenges/inner_merge.games_played

In [81]:
#total succesful challenges per team per game
inner_merge['tot_suc']=inner_merge.challenger2.apply(lambda x: len(inner_merge[(inner_merge.challenger2==x)&(inner_merge.overturned=='Yes')]))

In [82]:
#total successful challenges per team per game
inner_merge['tot_suc_p_g']=inner_merge.tot_suc/inner_merge.games_played

In [83]:
inner_merge.challenge_p_g.unique().mean()

0.23270331542202946

In [84]:
inner_merge.challenge_p_g.unique().max()+ucpg

NameError: name 'ucpg' is not defined

In [ ]:
(inner_merge.challenge_p_g.unique().max()+ucpg)/inner_merge.challenge_p_g.unique().mean()

In [ ]:
inner_merge.tot_suc_p_g.unique().mean()

In [ ]:
inner_merge.tot_suc_p_g.unique().max()+uspg

In [ ]:
(inner_merge.tot_suc_p_g.unique().max()+uspg)/inner_merge.tot_suc_p_g.unique().mean()

In [ ]:
#adjustment to take into account to convert to subset of challenged plays to all plays
gen_adj=((inner_merge.tot_suc_p_g.unique().max()+uspg)/inner_merge.tot_suc_p_g.unique().mean())/((inner_merge.challenge_p_g.unique().max()+ucpg)/inner_merge.challenge_p_g.unique().mean())

In [ ]:
#find the highest challenges per game (modeling later on) and highest succesful challenges per game
mcpg=max(inner_merge.challenge_p_g)
mspg=max(inner_merge.tot_suc_p_g)

In [ ]:
mspg/mcpg

In [ ]:
ump_plays=[x for x in inner_merge.play_type.unique() if x not in inner_merge[inner_merge.challenger2!='UMP'].play_type.unique()]

In [ ]:
tot_ump_c=len(inner_merge[(inner_merge.play_type.isin(ump_plays)==False)&(inner_merge.challenger2=='UMP')])/2

In [ ]:
tot_ump_s=len(inner_merge[(inner_merge.play_type.isin(ump_plays)==False)&(inner_merge.challenger2=='UMP')&(inner_merge.overturned=='Yes')])/2


In [ ]:
ucpg=float(tot_ump_c)/6517

In [ ]:
uspg=float(tot_ump_s)/6517

# Part III Calculate the "opportunity cost" of a lost challenge

In [ ]:
import mlbgame

In [ ]:
#winner function inputs a team and date and leverages mlbgame package to output if the team won that game
def winner(challenger,date):
    winner=mlbgame.day(date.year,date.month,date.day,home=inv_teams[challenger],away=inv_teams[challenger])
    if len(winner)<1:
        return 'Wrong Input'
    elif winner[0].w_team==inv_teams[challenger]:
        return 1
    else:
        return 0

In [ ]:
inner_merge['won_game']=inner_merge.apply(lambda row: winner(row['Tm'], row['Date']), axis=1)

In [ ]:
#finding winning percentage, given lost challenge
winning_pct_lost=inner_merge[(inner_merge.challenger2!='UMP')&(inner_merge.overturned=='No')].won_game.mean()

In [ ]:
#finding winning percentage, given won challenge
winning_pct_won=inner_merge[(inner_merge.challenger2!='UMP')&(inner_merge.overturned=='Yes')].won_game.mean()

In [ ]:
(.5-inner_merge[(inner_merge.challenger2!='UMP')&(inner_merge.overturned=='No')&(inner_merge.inning==7)].won_game.mean())-(inner_merge[(inner_merge.challenger2!='UMP')&(inner_merge.overturned=='Yes')&(inner_merge.inning==7)].won_game.mean()-.5)

In [ ]:
for i in range(1,10):
    print i, inner_merge[(inner_merge.challenger2!='UMP')&(inner_merge.overturned=='No')&(inner_merge.inning==i)].won_game.mean()

In [ ]:
ocpo=(.5-inner_merge[(inner_merge.challenger2!='UMP')&(inner_merge.overturned=='No')&(inner_merge.inning==1)].won_game.mean())/49

In [ ]:
#find the average number of innings remaining after a challenge
9-inner_merge[inner_merge.challenger2!='UMP'].inning.mean()

In [ ]:
#convert to number of outs
2.5*6+4

In [ ]:
opp_cost_per_out=(abs(.5-winning_pct_lost)-abs(.5-winning_pct_won))/19

In [ ]:
opp_cost_per_out

In [ ]:
(.5-inner_merge[(inner_merge.challenger2!='UMP')&(inner_merge.overturned=='No')&(inner_merge.inning==1)].won_game.mean())/49

In [ ]:
ocpo*46

# Testing Model

Inputs needed:
1. Team
2. day
3. umpire
4. play type
5. before challenge
 - Inning
 - Top/Bottom
 - Outs
 - Runners on What base
 - score differential
6. after challenge:
 - Inning
 - Top/Bottom
 - Outs
 - Runners on What base
 - score differential
7. team vs. general variable

In [ ]:
#logistic regression with gridsearch
def exp_val(team,day,umpire,play_type,inning1,inning2,half1,half2,outs1,outs2,fb1,fb2,sb1,sb2,tb1,tb2,dif1,dif2,weights=.5):

    #finds the potential gain of an overturned call in WPA
    gain=abs(win_exp(inning2,half2,fb2,sb2,tb2,outs2,dif2)-win_exp(inning1,half1,fb1,sb1,tb1,outs1,dif1))
    
    #transforms the input data into a vector to feed into the model
    vec=[challenger_dict[team][0],
    challenger_dict[team][1], 
    challenger_dict[team][2],
    umpire_dict[umpire][0],
    umpire_dict[umpire][1],
    umpire_dict[umpire][2],
    play_dict[play_type][0],
    play_dict[play_type][1],
    play_dict[play_type][2],
    day_dict[day][0],
    day_dict[day][1],
    day_dict[day][2],
    inning_dict[inning1][0],
    inning_dict[inning1][1],
    inning_dict[inning1][2]]
    
    #generates the probability of success, using team tendency
    prob=svm.predict_proba(vec)[0][1]
    
    #generates the probability of success based on general model
    prob2=(log_reg2.predict_proba(vec[3:])[0][1])*gen_adj
    
    #attaches weights to each of the two probabilities
    weighted_prob=weights*prob+(1-weights)*prob2
    
    #multiplies all together to get the expected upside of overturned call
    upside=gain*weighted_prob
    
    #find out how many outs remaining to project downside of call
    if half1==1:
        remaining=(9-inning1)*6+4+3+3-outs1
    else:
        remaining=(9-inning1)*6+4+3-outs1
    
    #downside is number of outs remaining times cost per out
    downside=(1-weighted_prob)*remaining*ocpo
    
    print 'Expected upside:', upside
    print 'Expected downside: ', downside
    print 'Net Gain: ', upside-downside
    return upside

In [ ]:
exp_val('TOR',1,'Angel Hernandez','Force play',1,1,1,1,1,0,0,1,0,0,0,0,0,0,0)

In [ ]:
svm.predict_proba([challenger_dict['NYN'][0],
    challenger_dict['NYN'][1], 
    challenger_dict['NYN'][2],
    umpire_dict["Joe West"][0],
    umpire_dict['Joe West'][1],
    umpire_dict['Joe West'][2],
    play_dict['Force play'][0],
    play_dict['Force play'][1],
    play_dict['Force play'][2],
    day_dict[2][0],
    day_dict[2][1],
    day_dict[2][2],
    inning_dict[1][0],
    inning_dict[1][1],
    inning_dict[1][2]])[0][1]

In [ ]:
inner_merge.play_type.unique()